## 1. Environment set-up

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
import supervision as sv
import os

## 2. Loading the dataset

In [3]:
dataset = os.listdir('../../images/best_images/original')
images = []

for paths in sorted(dataset):

  image_bgr = cv2.imread(f'../../images/best_images/original/{paths}')

  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

  data = {'name': paths,
        'bgr': image_bgr,
        'rgb': image_rgb}

  images.append(data)

print(f"O dataset contém {len(images)} imagens.")

O dataset contém 5 imagens.


## 3. Automatic mask generation

In [4]:
sam_checkpoint = "../../models/trained/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100)

print(f"Dispositivo utilizado: {device}")

Dispositivo utilizado: cuda:0


In [ ]:
annotaded_images = []
mask_annotator = sv.MaskAnnotator(color_lookup = sv.ColorLookup.INDEX)


output = len(os.listdir('/content/output/'))
print(f"A pasta de Output possui {output} imagens")

for image in images:

    sam_result = mask_generator.generate(image['rgb'])


    detections = sv.Detections.from_sam(sam_result=sam_result)
    annotaded_image = mask_annotator.annotate(scene=image['bgr'], detections=detections)

    # salvando a imagem
    cv2.imwrite(f'/content/output/{(image["name"])}', annotaded_image)

    annotaded_images.append(annotaded_image)

print(f'Foram gerados {len(annotaded_images)} resultados')